In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
drive.mount('/content/gdrive')

vectorizer = TfidfVectorizer(min_df=100, ngram_range=(1, 3))

Mounted at /content/gdrive


In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/train.csv')
val = pd.read_csv('/content/gdrive/MyDrive/val.csv')
train = train[train['Class']!='Human']

In [ ]:
fake_val = pd.read_csv('binary_predictions.csv', sep='\t')

In [ ]:
fake_val = fake_val[fake_val['predictions'] == 0]

In [ ]:
strings = []
for i, data in fake_val[:2].iterrows():
    strings.append(val[val['Text']==data[3]].values[0].tolist())

In [ ]:
pd.DataFrame(strings, columns=val.columns)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Id,Text,Class,Tokens,Lemmas,len in symbols,len in tokens,...,long words / all words,small words,Dale-Chall,Gunning-Fog,Flesch,func_pos/all_words,cont_pos/all_words,noun_pos/all_words,verb_pos/all_words,noun_pos/verb_pos
0,7046,7046,7046,71773,"Три построенных с данной целью здания, в том ч...",ruGPT3-Small,"['Три', 'построенных', 'с', 'данной', 'целью',...","['три', 'построить', 'с', 'дать', 'цель', 'зда...",641,90,...,0.018,0.026,19.95,7.12,104.17,0.222,0.678,0.578,0.1,5.778
1,10336,10336,10336,104119,обеспечение равных условий защищенности для вс...,Human,"['обеспечение', 'равных', 'условий', 'защищенн...","['обеспечение', 'равный', 'условие', 'защищённ...",76,9,...,0.029,0.009,19.87,3.60,113.10,0.111,0.889,0.889,0.0,0.000


In [ ]:
val[val['Text']==fake_val['texts'][0]]

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Id,Text,Class,Tokens,Lemmas,len in symbols,len in tokens,...,long words / all words,small words,Dale-Chall,Gunning-Fog,Flesch,func_pos/all_words,cont_pos/all_words,noun_pos/all_words,verb_pos/all_words,noun_pos/verb_pos
7046,7046,7046,7046,71773,"Три построенных с данной целью здания, в том ч...",ruGPT3-Small,"['Три', 'построенных', 'с', 'данной', 'целью',...","['три', 'построить', 'с', 'дать', 'цель', 'зда...",641,90,...,0.018,0.026,19.95,7.12,104.17,0.222,0.678,0.578,0.1,5.778


In [ ]:
from collections import Counter
from tqdm.notebook import tqdm
vocab = Counter()
for text in tqdm(train.Text):
    for symbol in text:
        if symbol is not "\n":
            vocab.update(list(symbol))
print('всего уникальных символов:', len(vocab))

In [ ]:
symbol2id = {'PAD':0}

for symbol in tqdm(vocab):
    symbol2id[symbol] = len(symbol2id)
    
id2symbol = {i:symbol for symbol, i in symbol2id.items()}

In [ ]:
train_transformed = vectorizer.fit_transform(train['Text']).toarray()
val_transformed = vectorizer.transform(val['Text']).toarray()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

In [ ]:
class TFIDFDataset(Dataset):
    def __init__(self, matrix, labels, DEVICE, features, symbol2id, dataset):
        self.matrix = matrix
        self.length = matrix.shape[0]
        self.target = labels
        self.device = DEVICE
        self.dataset = dataset['Text'].values

    def __len__(self): #это обязательный метод, он должен уметь считать длину датасета
        return self.length

    def __getitem__(self, index): #еще один обязательный метод. По индексу возвращает элемент выборки
        ids = torch.Tensor(self.matrix[index])
        y = self.target[index]
        texts = self.dataset[index]
        return ids, y, texts

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

train_labels = encoder.fit_transform(train['Class'])
val_labels = encoder.transform(val['Class'])

In [ ]:
train_data = TFIDFDataset(train_transformed, train_labels, 'cpu', train)
val_data = TFIDFDataset(val_transformed, val_labels, 'cpu',val)
train_iterator = DataLoader(train_data, shuffle=True, batch_size=64)
val_iterator = DataLoader(val_data, shuffle=True, batch_size=64)

In [ ]:
test_batch = next(iter(val_iterator))

In [ ]:
class MLP(nn.Module):
    
    def __init__(self, num_classes, vocab_size, embedding_dim=5374):
        
        super().__init__()          
        # указываем в атрибутах класса, какие слои и активации нам понадобятся
        self.emb2h = nn.Linear(embedding_dim, 300) 
        self.act1 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

        self.h2h = nn.Linear(300, 100)
        self.h2out = nn.Linear(100, num_classes)
        
        
    def forward(self, text, features, symbols): #необходимый метод,  в нем указываем, как именно связываются слои/активации между собой
        # batch_size x seq_len
        #ipdb.set_trace()
        hidden = self.emb2h(text) # пропускаем эмбеддинг через полносвязный слой 
        hidden = self.act1(hidden)
        hidden = self.dropout(hidden)
        hidden = self.act1(self.h2h(hidden))
        # batch_size x 10
        out = self.h2out(hidden) # возвращаем одно число для каждого семпла
        # batch_size x 1# пропускаем число через сигмоиду, делая из него вероятность класса
        # batch_size x 1
        return out

In [ ]:
def train(model, iterator, optimizer, criterion):
    print('Training...')
    epoch_loss = 0 # для подсчета среднего лосса на всех батчах
    model.train()  # ставим модель в обучение, явно указываем, что сейчас надо будет хранить градиенты у всех весов

    for i, (tfidf ys, texts) in enumerate(iterator): #итерируемся по батчам
        tfidf = tfidf.to('cuda:0')
        ys = ys.to('cuda:0')
        optimizer.zero_grad()  #обнуляем градиенты
        #ipdb.set_trace()
        preds_proba = model(tfidf) #прогоняем данные через модель
        loss = criterion(preds_proba, ys) #считаем значение функции потерь  
        loss.backward() #считаем градиенты  
        optimizer.step() #обновляем веса 
        epoch_loss += loss.item() #сохраняем значение функции потерь
        
    print(f'Train loss: {epoch_loss/i}')
    return epoch_loss / len(iterator) # возвращаем среднее значение функции потерь по всей выборке

In [ ]:
def evaluate(model, iterator, criterion):
    print("\nValidating...")
    epoch_loss = 0
    accuracy = []
    model.eval() 
    text = []
    pred = []
    label = []
    with torch.no_grad():
        for i, (tfidf,  ys, texts) in enumerate(iterator):   
            tfidf = tfidf.to('cuda:0')
            ys = ys.to('cuda:0')
            predictions = model(tfidf)  # делаем предсказания на тесте
            loss = criterion(predictions, ys)   # считаем значения функции ошибки для статистики  
            epoch_loss += loss.item() 
            accuracy.append(accuracy_score(ys.cpu().detach().numpy(), predictions.argmax(axis=1).cpu().detach().numpy()))
            label.extend(ys.cpu().detach().tolist())
            pred.extend(predictions.argmax(axis=1).cpu().detach().tolist())
            text.extend(list(texts))
    print(sum(accuracy)/len(accuracy))
    return epoch_loss / len(iterator), text, pred, label # возвращаем средний лосс по батчам

In [ ]:
from torch import optim

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
model = MLP(num_classes=13, vocab_size=len(symbol2id), embedding_dim=3358)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss() # Binary Cross Entropy



# веса модели и значения лосса храним там же, где и все остальные тензоры
model = model.to('cuda:0')
criterion = criterion.to('cuda:0')

In [ ]:
losses = []
losses_eval = []

for i in tqdm(range(15)):
    print(f'\nstarting Epoch {i}')
    epoch_loss = train(model, train_iterator, optimizer, criterion)
    losses.append(epoch_loss)

    epoch_loss_on_test = evaluate(model, val_iterator, criterion)
    losses_eval.append(epoch_loss_on_test)

In [ ]:
epoch_loss_on_test, texts, predictions, labels = evaluate(model, val_iterator, criterion)
df = pd.DataFrame({'texts':texts, 'predictions':encoder.inverse_transform(predictions), 
                   'labels':encoder.inverse_transform(labels)})
df.to_csv('combo_model.csv')